## Import Dependencies

In [1]:
import pandas as pd
import requests
import pymongo
import os
import tweepy
import json
import numpy as np

# from config import consumer_key, consumer_secret, access_token, access_token_secret
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from pprint import pprint

consumer_key = "36Ir92uhNmDnv6Sn6tr2BicdY"
consumer_secret = "URccAlZmOS3COMVspRM92dla9lNi76NpbUMhrdEUOmH5OWLcvO"
access_token = "1038858933560115200-NgoqDCmZUVm9Z41KWN89s1MrB2tegs"
access_token_secret = "l3kG1sk2mCWWTaZIdC7RDoJZxN1iWZ3JdYXYf3XyItqfR"
# Setup Tweepy Authentication 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

## Step 1 - Scraping

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
db = client.space_db
collection = db.articles

## NASA Mars News

In [ ]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [ ]:
# Retrieve page with the requests module
response = requests.get(url)

# Create BeautifulSoup object; parse with 'lxml'
soup = bs(response.text, 'lxml')

In [ ]:
results = soup.find_all("div", class_="list_text")

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

posts = soup.find_all("div", class_="list_text")

title = []
para = []

for post in posts:
    title.append(post.a.text)
    para.append(post.find("div", class_="article_teaser_body").text)

    article ={
        "news_title": title[0],
        "news_text": para[0]
    }

#     news_title = title[0]
#     news_p = para[0]

In [ ]:
print(article["news_title"])
print(article["news_text"])

In [ ]:
for result in results:
    title = result.a.text
    para = result.find("div", class_="article_teaser_body").text
    
    article = {
        "news_title": title,
        "news_paragraph": para
    }
    
    print("---------------------------------")
    print(f'{title}')
    print(f'{para}')

## Finding the Images URL

In [ ]:
url2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url2)

In [ ]:
# Retrieve page with the requests module
response_image = requests.get(url2)

# Create BeautifulSoup object; parse with 'lxml'
soup_image = bs(response_image.text, 'lxml')

In [ ]:
browser.click_link_by_partial_text('FULL IMAGE')

In [ ]:
browser.click_link_by_partial_text('more info')

In [ ]:
# URL varies depending on what the web browser opens
url_image = "https://www.jpl.nasa.gov/spaceimages/details.php?id=PIA17924"

In [ ]:
# Retrieve page with the requests module
response_image1 = requests.get(url_image)

# Create BeautifulSoup object; parse with 'lxml'
soup_image1 = bs(response_image1.text, 'lxml')

In [ ]:
results_image = soup_image1.find_all("figure", class_="lede")

In [ ]:
# Scarping the image url
image_url = []
for image in results_image:
    image_url.append(image.a["href"])

In [ ]:
image_url

In [ ]:
# Adding the url into images
for url in image_url:
    featured_image_url= "https://www.jpl.nasa.gov"+url
    print(featured_image_url)

## Mars Weather

In [5]:
target_user = "@MarsWxReport"

In [6]:
public_tweets = api.user_timeline(target_user)

In [7]:
weather_tweet = []

for tweet in public_tweets:
    weather_tweet.append(tweet["text"])

In [8]:
mars_weather = weather_tweet[0]
print(mars_weather)

Sol 2242 (2018-11-26), high -2C/28F, low -70C/-93F, pressure at 8.48 hPa, daylight 06:29-18:45


## Mars Facts

In [9]:
url_facts = "https://space-facts.com/mars/"

In [10]:
tables = pd.read_html(url_facts)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [11]:
df = tables[0]

In [12]:
df_clean = df.set_index(0)
df_clean.index.name=None

In [13]:
df_clean = df_clean.rename(columns={1: "Values"})
df_clean

,Values
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [14]:
html_table = df_clean.to_html()

In [17]:
mars_table = html_table.replace('\n', '')

In [19]:
mars_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [16]:
df_clean.to_html('mars_facts.html')

## Mars Hemisphere

In [21]:
url_cerberus = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
url_schiaparelli = "https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced"
url_syrtis = "https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced"
url_valles = "https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced"

In [77]:
url_hemisphere = [url_cerberus, url_schiaparelli, url_syrtis, url_valles]

for url in url_hemisphere:
    response_hemisphere = requests.get(url)
    soup_hemisphere = bs(response_hemisphere.text, 'lxml')
    
    results_hemisphere = soup_hemisphere.find_all("div", class_="container")
    
    for result in results_hemisphere:
        title = result.h2.text
        img_url = result.find("img", class_="wide-image")["src"]
        
        hemisphere_image_urls = {
            "title": title,
            "img_url": "https://astrogeology.usgs.gov"+img_url
        }
        
        print(hemisphere_image_urls)

{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}


In [76]:
lol = testing[0]
hehe = testing[1]

print(lol)
print(hehe)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg


In [50]:
data = {
    "img_url": [img["img_url"] for img in testing]
} 

In [57]:
testing[0]["img_url"]

'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'

In [53]:
for test in data["img_url"]:
    print(test)

https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg
